In [2]:
let mut s = String::new();


In [3]:
let data = "initial contents";
// 使用 to_string 方法从字符串字面值创建 String
let s = data.to_string();

// 该方法也可直接用于字符串字面值：
let s = "initial contents".to_string();

s

"initial contents"

因为字符串应用广泛，这里有很多不同的用于字符串的通用 API 可供选择。其中一些可能看起来多余，不过都有其用武之地！在这个例子中，String::from 和 .to_string 最终做了完全相同的工作，所以如何选择就是风格问题了。



In [ ]:
// 使用 String::from 函数来从字符串字面值创建 String
let s = String::from("initial contents");


In [4]:
// 记住字符串是 UTF-8 编码的，所以可以包含任何可以正确编码的数据
let hello = String::from("السلام عليكم");
let hello = String::from("Dobrý den");
let hello = String::from("Hello");
let hello = String::from("שָׁלוֹם");
let hello = String::from("नमस्ते");
let hello = String::from("こんにちは");
let hello = String::from("안녕하세요");
let hello = String::from("你好");
let hello = String::from("Olá");
let hello = String::from("Здравствуйте");
let hello = String::from("Hola");

String 的大小可以增加，其内容也可以改变，就像可以放入更多数据来改变 Vec 的内容一样。另外，可以方便的使用 + 运算符或 format! 宏来拼接 String 值。



In [5]:
let mut s = String::from("foo");
s.push_str("bar");
s

"foobar"

In [6]:
let mut s1 = String::from("foo");
let s2 = "bar";
// push_str 方法采用字符串 slice，因为我们并不需要获取参数的所有权。
s1.push_str(s2);
println!("s2 is {}", s2);

s2 is bar


In [7]:
// push 方法被定义为获取一个单独的字符作为参数，并附加到 String 中
let mut s = String::from("lo");
s.push('l');
s

"lol"

In [10]:
let s1 = String::from("Hello, ");
let s2 = String::from("world!");
let s3 = s1 + &s2; // 注意 s1 被移动了，不能继续使用
(s3, s2)

("Hello, world!", "world!")

这是因为 add 函数的 s 参数：只能将 &str 和 String 相加，不能将两个 String 值相加。不过等一下 —— 正如 add 的第二个参数所指定的，&s2 的类型是 &String 而不是 &str。
之所以能够在 add 调用中使用 &s2 是因为 &String 可以被 强转（coerced）成 &str。当add函数被调用时，Rust 使用了一个被称为 解引用强制多态（deref coercion）的技术，你可以将其理解为它把 &s2 变成了 &s2[..]。

In [11]:
// 如果想要级联多个字符串，+ 的行为就显得笨重了：

let s1 = String::from("tic");
let s2 = String::from("tac");
let s3 = String::from("toe");

let s = s1 + "-" + &s2 + "-" + &s3;
s

"tic-tac-toe"

In [12]:
// 可以使用 format! 宏：

let s1 = String::from("tic");
let s2 = String::from("tac");
let s3 = String::from("toe");

let s = format!("{}-{}-{}", s1, s2, s3);
s

"tic-tac-toe"

In [13]:
// Rust 的字符串不支持索引
let s1 = String::from("hello");
let h = s1[0];

Error: the type `std::string::String` cannot be indexed by `{integer}`

In [14]:
let len = String::from("Hola").len();
len

4

当问及这个字符是多长的时候有人可能会说是 12。然而，Rust 的回答是 24。这是使用 UTF-8 编码 “Здравствуйте” 所需要的字节数，这是因为每个 Unicode 标量值需要两个字节存储。因此一个字符串字节值的索引并不总是对应一个有效的 Unicode 标量值。

In [15]:
let len = String::from("Здравствуйте").len();
len

24

```rust
let hello = "Здравствуйте";
let answer = &hello[0];
```
answer 的值应该是什么呢？它应该是第一个字符 З 吗？当使用 UTF-8 编码时，З 的第一个字节 208，第二个是 151，所以 answer 实际上应该是 208，不过 208 自身并不是一个有效的字母。返回 208 可不是一个请求字符串第一个字母的人所希望看到的，不过它是 Rust 在字节索引 0 位置所能提供的唯一数据。用户通常不会想要一个字节值被返回，即便这个字符串只有拉丁字母： 即便 &"hello"[0] 是返回字节值的有效代码，它也应当返回 104 而不是 h。为了避免返回意外的值并造成不能立刻发现的 bug，Rust 根本不会编译这些代码，并在开发过程中及早杜绝了误会的发生。

### 字节、标量值和字形簇！天呐！
这引起了关于 UTF-8 的另外一个问题：从 Rust 的角度来讲，事实上有三种相关方式可以理解字符串：字节、标量值和字形簇（最接近人们眼中 字母 的概念）。

比如这个用梵文书写的印度语单词 “नमस्ते”，最终它储存在 vector 中的 u8 值看起来像这样：

[224, 164, 168, 224, 164, 174, 224, 164, 184, 224, 165, 141, 224, 164, 164, 224, 165, 135]

这里有 18 个字节，也就是计算机最终会储存的数据。如果从 Unicode 标量值的角度理解它们，也就像 Rust 的 char 类型那样，这些字节看起来像这样：

['न', 'म', 'स', '्', 'त', 'े']

这里有六个 char，不过第四个和第六个都不是字母，它们是发音符号本身并没有任何意义。最后，如果以字形簇的角度理解，就会得到人们所说的构成这个单词的四个字母：

["न", "म", "स्", "ते"]

Rust 提供了多种不同的方式来解释计算机储存的原始字符串数据，这样程序就可以选择它需要的表现方式，而无所谓是何种人类语言。

最后一个 Rust 不允许使用索引获取 String 字符的原因是，索引操作预期总是需要常数时间 (O(1))。但是对于 String 不可能保证这样的性能，因为 Rust 必须从开头到索引位置遍历来确定有多少有效的字符。

In [16]:
// 字符串 slice
let hello = "Здравствуйте";

let s = &hello[0..4];
s

"Зд"

### 遍历字符串的方法
幸运的是，这里还有其他获取字符串元素的方式。

如果你需要操作单独的 Unicode 标量值，最好的选择是使用 chars 方法。对 “नमस्ते” 调用 chars 方法会将其分开并返回六个 char 类型的值，接着就可以遍历其结果来访问每一个元素了：

In [17]:
for c in "नमस्ते".chars() {
    println!("{}", c);
}

न
म
स
्
त
े


()

In [18]:
// bytes 方法返回每一个原始字节，这可能会适合你的使用场景：

for b in "नमस्ते".bytes() {
    println!("{}", b);
}

224
164
168
224
164
174
224
164
184
224
165
141
224
164
164
224
165
135


()

总而言之，字符串还是很复杂的。不同的语言选择了不同的向程序员展示其复杂性的方式。Rust 选择了以准确的方式处理 String 数据作为所有 Rust 程序的默认行为，这意味着程序员们必须更多的思考如何预先处理 UTF-8 数据。这种权衡取舍相比其他语言更多的暴露出了字符串的复杂性，不过也使你在开发生命周期后期免于处理涉及非 ASCII 字符的错误。

